In [ ]:
import arff
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import theano.tensor as tt

In [ ]:
x_column = 'gamma-log'
y_column = 'score-at-task-3'
x_idx = None
y_idx = None

with open('/home/janvanrijn/projects/openml-multitask/data/svm-gamma-10tasks.arff', 'r') as fp:
    dataset = arff.load(fp)
    dataset['data'] = np.array(dataset['data'], dtype=np.float)

for idx, (column, _) in enumerate(dataset['attributes']):
    if column == x_column:
        x_idx = idx
    elif column == y_column:
        y_idx = idx

X0 = dataset['data'][:,x_idx].reshape(-1, 1)
f = dataset['data'][:,y_idx]

n = X0.shape[0]
X0_min = min(X0)[0]
X0_max = max(X0)[0]

fig, ax = plt.subplots(figsize=(14, 6));
ax.scatter(X0, f, s=40, color='b', label='observations');

In [ ]:
np.random.seed(1)

# Number of points at which to interpolate
m = 100
X = np.linspace(X0_min, X0_max, m)[:, None]

# Covariance kernel parameters
noise = 0.01
lengthscale = 1.0
f_scale = 1

cov = f_scale * pm.gp.cov.ExpQuad(1, lengthscale)
K = cov(X0)
K_s = cov(X0, X)
K_noise = K + noise * tt.eye(n)

# Add very slight perturbation to the covariance matrix diagonal to improve numerical stability
K_stable = K + 1e-12 * tt.eye(n)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6));
ax.scatter(X0, f, s=40, color='b', label='True points');

# Analytically compute posterior mean
L = np.linalg.cholesky(K_noise.eval())
alpha = np.linalg.solve(L.T, np.linalg.solve(L, f))
post_mean = np.dot(K_s.T.eval(), alpha)

ax.plot(X, post_mean, color='g', alpha=0.8, label='Posterior mean');

ax.set_xlim(X0_min, X0_max);
ax.set_ylim(0, 1);
ax.legend();

In [ ]:
# JvR: This cell often crashes the first time when it's executed. In order to overcome this,
# rerun from cell 3 and on
# TODO: Figure out WHY? 
with pm.Model() as model:
    # The actual distribution of f_sample doesn't matter as long as the shape is right since it's only used
    # as a dummy variable for slice sampling with the given prior
    f_sample = pm.Flat('f_sample', shape=(n, ))

    # Likelihood
    y = pm.MvNormal('y', observed=f, mu=f_sample, cov=noise * tt.eye(n), shape=n)

    # Interpolate function values using noisy covariance matrix
    L = tt.slinalg.cholesky(K_noise)
    f_pred = pm.Deterministic('f_pred', tt.dot(K_s.T, tt.slinalg.solve(L.T, tt.slinalg.solve(L, f_sample))))

    # Use elliptical slice sampling
    ess_step = pm.EllipticalSlice(vars=[f_sample], prior_cov=K_stable)
    trace = pm.sample(5000, start=model.test_point, step=[ess_step], progressbar=False, random_seed=1)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6));
for idx in np.random.randint(4000, 5000, 500):
    ax.plot(X, trace['f_pred'][idx],  alpha=0.02, color='navy')
ax.scatter(X0, f, s=40, color='k', label='True points');
ax.plot(X, post_mean, color='g', alpha=0.8, label='Posterior mean');
ax.legend();
ax.set_xlim(X0_min, X0_max);
ax.set_ylim(0, 1);